In [36]:
import pandas as pd
import numpy as np
from pathlib import Path

In [51]:
years = np.arange(2018, 2024)
months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
path_base = Path('data')
directories = [path_base / str(year) / month for year in years for month in months]

dfs = []
for dir in directories:
    if dir.exists():
        for file in dir.iterdir():
            if file.is_file() and file.name.startswith('y'):
                df = wrangle(file)
                dfs.append(df)
len(dfs)



2191

In [104]:
dfs[0]['Date'] =  dfs[0].iloc[0]['Parameter']
df['Parameter'].replace('', np.nan, inplace=True)

# Forward fill the 'Parameter' column
df['Parameter'] = df['Parameter'].ffill()


array(['1/1/2018', '', 'Parameter', '01 OZONE PPM', '02 NOX PPM',
       '03 NO2 PPM', '04 NO PPM', '06 CO PPM', '10 INTMP °C',
       '11 PM2.5 µg/M3', '12 VWDR °', '13 SIGTHETA °', '14 VWSP MPH',
       '15 SWSP MPH', '16 AMBTEMP °F', '17 RELHUM %', '18 PRES mmHg',
       '19 SOLAR W/M2', '28 SO2 Tr PPB', '29 CO Tr PPB'], dtype=object)

In [67]:
biggie = pd.concat(dfs)
# Convert 'Avg' column to numeric, coercing errors to NaN
biggie['Avg'] = pd.to_numeric(biggie['Avg'], errors='coerce')

# Now group by 'Site Name' and compute the mean
result = biggie[['Site Name', 'Avg']].iloc[3:].groupby('Site Name').mean()

result

,Avg
Site Name,
,NaN
10TH AVE MARINE TERM,0.795430
ALPINE,20.941639
BOSTON AVE,0.801569
CARMEL MT RCH,17.619119
CHICANO PARK,0.735259
CHULA VISTA,19.927455
DEL_MAR,NaN
DOWNTOWN,NaN


In [84]:
biggie.iloc[0:20]

,Parameter,Site Name,0,1,2,3,4,5,6,7,...,17,18,19,20,21,22,23,Avg,Max,Hr. of Max
0,1/1/2018,,,,,,,,,,...,,,,,,,,NaN,,
1,,,0,1,2,3,4,5,6,7,...,17,18,19,20,21,22,23,NaN,,
2,Parameter,SiteName,,,,,,,,,...,,,,,,,,NaN,Max,Hr. of Max
3,01 OZONE PPM,ALPINE,.049,C,C,.047,.048,.045,.043,.043,...,.045,.044,.042,.042,.044,.042,.046,0.045,.056,14
4,,CHULA VISTA,.002,C,C,.002,.004,.004,.002,.003,...,.033,.021,.011,.003,.004,.012,.016,0.019,.048,13
5,,DEL_MAR,,,,,,,,,...,,,,,,,,NaN,,
6,,DOWNTOWN,,,,,,,,,...,,,,,,,,NaN,,
7,,EL CAJON LES,.004,C,C,C,.003,.003,.003,.004,...,.028,.019,.009,.006,.004,.003,.003,0.020,.052,14
8,,ESCONDIDO,,,,,,,,,...,,,,,,,,NaN,,
9,,KEARNY MESA,.019,C,C,.017,.018,.018,.016,.010,...,.035,.028,.022,.020,.015,.007,.019,0.027,.050,14


In [50]:
import pandas as pd

# Read the file with custom parsing
file_path = '8_yesterday_20240917.csv'
def wrangle(file_path):
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            lines = file.readlines()
    except UnicodeDecodeError:
        with open(file_path, 'r', encoding='latin-1') as file:
            lines = file.readlines()
    
    # Extract date and irrelevant lines
    date_info = lines.pop(0).strip()
    lines.pop(0)
    
    # Initialize data structure
    data = []

    # Parse the remaining lines
    for line in lines:
        split_line = line.strip().split(',')
        if len(split_line) >= 29:  # Ensure there are at least 29 fields
            data.append(split_line[:29])  # Adjust if different number of fields per row

    # Create DataFrame from the parsed data
    columns = [
        'Parameter', 'Site Name', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16',
        '17', '18', '19', '20', '21', '22', '23', 'Avg', 'Max', 'Hr. of Max'
    ]
    df = pd.DataFrame(data, columns=columns)
    return df


In [28]:
geo = pd.read_csv("2015_San_Diego_County_Demographics_-_County_Population_by_Age_and_Gender.csv")
geo[['Geography', 'Percent 65+']]


,Geography,Percent 65+
0,Central Region,9.6
1,Central San Diego,10.3
2,Mid-City,8.1
3,Southeastern San Diego,10.2
4,East Region,13.3
5,Alpine,18.5
6,El Cajon,12.3
7,Harbison Crest,18.6
8,Jamul,13.1
9,La Mesa,15.0


In [ ]:
df.merge(geo, left_on='')